<a href="https://colab.research.google.com/github/Leasteep/BonScanner/blob/master/bonscanner_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.activations import relu, softmax
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.optimizers import Adam
from keras.models import Sequential
import glob
import os

from keras.optimizers import Adam
from keras.utils import np_utils
import imageio
import numpy as np
from sklearn.model_selection import StratifiedKFold



In [9]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')
!tar -xf 'drive/My Drive/Colab Notebooks/data/EnglishFnt.tar'

^C


In [0]:
images = []
labels = []
label_incrementor = 0
for directoryname in glob.glob('English/Fnt/Sample*'):
    for filename in glob.glob(os.path.join(directoryname, '*.png')):
        img = imageio.imread(filename)
        images.append(img)
        labels.append(np_utils.to_categorical(label_incrementor, num_classes=64))
    label_incrementor += 1
images = np.array(images)
images = np.expand_dims(images, 3)
labels = np.array(labels)

In [0]:
dropout = 0.5

cnn = Sequential()
cnn.add(Conv2D(input_shape=(128, 128, 1),
               filters=32,
               kernel_size=(5, 5),
               strides=(1, 1),
               activation=relu,
               padding='same'))
cnn.add(BatchNormalization())
cnn.add(MaxPooling2D())
cnn.add(Conv2D(filters=64,
               kernel_size=(5, 5),
               strides=(1, 1),
               activation=relu,
               padding='same'))
cnn.add(BatchNormalization())
cnn.add(MaxPooling2D())
cnn.add(Flatten())
cnn.add(Dropout(dropout))
cnn.add(Dense(units=128, activation=relu))
cnn.add(Dropout(dropout))
cnn.add(Dense(units=64, activation=softmax))

In [0]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

optimizer = Adam(lr=0.1)
callbacks = [ModelCheckpoint('drive/My Drive/Colab Notebooks/data/models/cnn_{epoch:02d}-{loss:.2f}.hdf5', 
                             monitor='loss', save_best_only=True),
             CSVLogger('drive/My Drive/Colab Notebooks/data/cvscores/cvscore.log')]


# define 10-fold cross validation test harness
# StratifiedKFold expects array of shape (samples, ) to calculate indices
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
i = 0 
for train, test in kfold.split(images, np.zeros(labels.shape[0], )):
    
    cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    cnn.fit(images[train], labels[train], epochs=60, batch_size=32, verbose=1, callbacks=callbacks)
    cnn.save('cnn_fold%s.h5' % i)
    
    scores = cnn.evaluate(images[test], labels[test], verbose=1)
    print("%s: %.2f%%" % (cnn.metrics_names[1], scores[1] * 100))
    cvscores.append(scores[1] * 100)
    np.save(cvscores, 'cvscores')
    i += 1

Epoch 1/60
56692/56692 [==============================] - 113s 2ms/step - loss: 2.2610 - acc: 0.3789
Epoch 2/60
56692/56692 [==============================] - 111s 2ms/step - loss: 1.9509 - acc: 0.4431
Epoch 3/60
56692/56692 [==============================] - 111s 2ms/step - loss: 1.4690 - acc: 0.5603
Epoch 4/60
56692/56692 [==============================] - 111s 2ms/step - loss: 1.0189 - acc: 0.6780
Epoch 5/60
56692/56692 [==============================] - 111s 2ms/step - loss: 0.7916 - acc: 0.7437
Epoch 6/60
56692/56692 [==============================] - 111s 2ms/step - loss: 0.6767 - acc: 0.7762
Epoch 7/60
56692/56692 [==============================] - 111s 2ms/step - loss: 0.6004 - acc: 0.7971
Epoch 8/60
56692/56692 [==============================] - 111s 2ms/step - loss: 0.5493 - acc: 0.8109
Epoch 9/60
56692/56692 [==============================] - 111s 2ms/step - loss: 0.5046 - acc: 0.8234
Epoch 10/60
55360/56692 [============================>.] - ETA: 2s - loss: 0.4752 - acc: 0.